## Setup

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import numpy as np

from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_parquet("../data/processed/abt_airline_reviews.parquet")
df.head(2)

,airline_name,overall_rating,review_title,review_date,review,aircraft,type_of_traveller,seat_type,route,date_flown,seat_comfort,cabin_staff_service,food_e_beverages,ground_service,inflight_entertainment,wifi_e_connectivity,review_clean,review_title_clean,review_day_of_week,date_flown_day_of_week,review_year_month,date_flown_year_month,review_year,date_flown_year,review_month,date_flown_month,days_difference,rating_category,rating_category_numeric,full_review_text,is_delay
0,Air Seychelles,8,Air Seychelles customer review,2011-04-10,LHR-SEZ-LHR. It's true that there aren't perso...,None,None,Economy Class,None,NaT,NaN,NaN,NaN,NaN,NaN,NaN,lhrsezlhr true not personal tv like virgin ba ...,air seychelle customer review,Sunday,None,2011-04,NaT,2011.0,NaN,4.0,NaN,NaN,Positivo,1,Air Seychelles customer review LHR-SEZ-LHR. It...,0
1,CityJet,1,CityJet customer review,2011-04-10,London City to Antwerp was delayed 3 hours 40 ...,None,None,Economy Class,None,NaT,NaN,NaN,NaN,NaN,NaN,NaN,london city antwerp delay hour min sure tr...,cityjet customer review,Sunday,None,2011-04,NaT,2011.0,NaN,4.0,NaN,NaN,Negativo,-1,CityJet customer review London City to Antwerp...,1


## Processamento dos dados

In [3]:
threshold = 0.6 
missing_ratio = df.isnull().mean()
cols_to_drop = missing_ratio[missing_ratio > threshold].index
df = df.drop(columns=cols_to_drop)

print(f'Colunas removidas por excesso de nulos (>60%): {list(cols_to_drop)}')

Colunas removidas por excesso de nulos (>60%): ['aircraft', 'wifi_e_connectivity']


## Configuração em comum entre os modelos

In [4]:
y = df['rating_category_numeric']

In [5]:
len(y)

17367

## Model 1 - Input Texto

In [6]:
tfidf = TfidfVectorizer(max_features=1000, stop_words='english')
X_text = tfidf.fit_transform(df['full_review_text'])

In [7]:
# Aplica SMOTE nos dados vetorizados
smote = SMOTE(random_state=42)
X_text_res, y_res = smote.fit_resample(X_text, y)

In [8]:
# Divisão em treino e teste
X_train_text, X_test_text, y_train, y_test = train_test_split(X_text_res, y_res, test_size=0.3, random_state=42)

# Modelo com Logistic Regression
model_text = LogisticRegression(max_iter=200)
model_text.fit(X_train_text, y_train)
y_pred_text = model_text.predict(X_test_text)

print("🔹 Avaliação do modelo com texto:")
print(classification_report(y_test, y_pred_text))

🔹 Avaliação do modelo com texto:
              precision    recall  f1-score   support

          -1       0.81      0.77      0.79      3657
           0       0.73      0.73      0.73      3697
           1       0.79      0.82      0.80      3707

    accuracy                           0.78     11061
   macro avg       0.78      0.78      0.78     11061
weighted avg       0.78      0.78      0.78     11061



In [9]:
# Validação cruzada com diferentes modelos e SMOTE dentro do pipeline
models_text = {
    'Logistic Regression': LogisticRegression(max_iter=200),
    'Random Forest': RandomForestClassifier(),
    'Naive Bayes': MultinomialNB()
}

print("🔹 Avaliação com dados textuais (TF-IDF + SMOTE):\n")
for name, model in models_text.items():
    pipeline = ImbPipeline([
        ('tfidf', TfidfVectorizer(max_features=1000, stop_words='english')),
        ('smote', SMOTE(random_state=42)),
        ('clf', model)
    ])
    scores = cross_val_score(pipeline, df['full_review_text'], y, 
                             cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42), 
                             scoring='accuracy')
    print(f'{name}: média = {scores.mean():.3f} | desvio padrão = {scores.std():.3f}')

🔹 Avaliação com dados textuais (TF-IDF + SMOTE):

Logistic Regression: média = 0.718 | desvio padrão = 0.009
Random Forest: média = 0.746 | desvio padrão = 0.008
Naive Bayes: média = 0.716 | desvio padrão = 0.006


## Model 2 - Inputs Numéricos

In [10]:
feature_cols = [
    'seat_comfort', 'cabin_staff_service', 'food_e_beverages',
    'ground_service', 'inflight_entertainment'
]

X_scores = df[feature_cols].astype(float).fillna(df[feature_cols].mean())


In [11]:
# Aplica SMOTE
smote = SMOTE(random_state=42)
X_scores_res, y_res = smote.fit_resample(X_scores, y)

In [12]:
# Divisão em treino e teste
X_train_scores, X_test_scores, y_train, y_test = train_test_split(X_scores_res, y_res, test_size=0.3, random_state=42)

# Padronização dos dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_scores)
X_test_scaled = scaler.transform(X_test_scores)

# Modelo com Logistic Regression
model_scores = LogisticRegression(max_iter=200)
model_scores.fit(X_train_scaled, y_train)
y_pred_scores = model_scores.predict(X_test_scaled)

print("🔹 Avaliação do modelo com notas:")
print(classification_report(y_test, y_pred_scores))

🔹 Avaliação do modelo com notas:
              precision    recall  f1-score   support

          -1       0.64      0.78      0.70      3657
           0       0.70      0.43      0.53      3697
           1       0.68      0.80      0.74      3707

    accuracy                           0.67     11061
   macro avg       0.68      0.67      0.66     11061
weighted avg       0.68      0.67      0.66     11061



In [13]:
# Avaliação com validação cruzada + SMOTE + Scaler para todos os modelos
models_scores = {
    'Logistic Regression': LogisticRegression(max_iter=200),
    'Random Forest': RandomForestClassifier(),
    'Naive Bayes': GaussianNB()
}

print("\n🔹 Avaliação com notas (features numéricas):\n")
for name, model in models_scores.items():
    pipeline = ImbPipeline([
        ('scaler', StandardScaler()),
        ('smote', SMOTE(random_state=42)),
        ('clf', model)
    ])
    scores = cross_val_score(pipeline, X_scores, y, 
                             cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42),
                             scoring='accuracy')
    print(f'{name}: média = {scores.mean():.3f} | desvio padrão = {scores.std():.3f}')


🔹 Avaliação com notas (features numéricas):

Logistic Regression: média = 0.731 | desvio padrão = 0.010
Random Forest: média = 0.728 | desvio padrão = 0.018
Naive Bayes: média = 0.680 | desvio padrão = 0.006


## Impacto de atrasos no NPS

In [14]:
def nps_result(top_airlines, df, model_text=model_text, tfidf=tfidf):
    nps_resultados = []

    for airline in top_airlines:
        df_comp = df[df['airline_name'] == airline]
        
        for status, label in zip([0, 1], ['Sem Atraso', 'Com Atraso']):
            subset = df_comp[df_comp['is_delay'] == status]
            
            # Preprocessamento básico (se necessário)
            textos = (subset['review'].fillna('') + ' ' + subset['review_title'].fillna('')).tolist()
            X_text = tfidf.transform(textos)
            
            # Previsão de sentimento
            preds = model_text.predict(X_text)

            # Calcular % positivos e % negativos
            positivos = np.sum(preds == 1)
            negativos = np.sum(preds == -1)
            total = positivos + negativos

            if total == 0:
                nps = np.nan
            else:
                nps = (positivos / total - negativos / total) * 100
            
            nps_resultados.append({
                'Companhia': airline,
                'Atraso': label,
                'NPS': round(nps, 2)
            })

    # Exibir como DataFrame
    nps_df = pd.DataFrame(nps_resultados)
    print(nps_df)

### Seleção de 3 companhias aéreas - mais frequentes

In [15]:
top_airlines = df['airline_name'].value_counts().head(3).index

print("Companhias aéreas mais frequentes:")
print(top_airlines)

Companhias aéreas mais frequentes:
Index(['Oman Air', 'EVA Air', 'Air Arabia'], dtype='object', name='airline_name')


In [16]:
nps_result(top_airlines, df)

    Companhia      Atraso     NPS
0    Oman Air  Sem Atraso  -27.27
1    Oman Air  Com Atraso  -80.95
2     EVA Air  Sem Atraso   35.00
3     EVA Air  Com Atraso  -22.58
4  Air Arabia  Sem Atraso  -78.95
5  Air Arabia  Com Atraso -100.00


In [17]:
print("Avaliações negativas: ", len(df[(df['airline_name'].isin(top_airlines)) & (df['rating_category_numeric'] == -1)]))
print("Avaliações positivas: ", len(df[(df['airline_name'].isin(top_airlines)) & (df['rating_category_numeric'] == 1)]))
print("Avaliações neutras: ", len(df[(df['airline_name'].isin(top_airlines)) & (df['rating_category_numeric'] == 0)]))

Avaliações negativas:  218
Avaliações positivas:  43
Avaliações neutras:  39


### Seleção de 3 companhias aéreas - avaliação +negativas

In [18]:
top_negative_airlines = df[df.rating_category=='Negativo']['airline_name'].value_counts().head(3).index

print("Companhias aéreas mais frequentes:")
print(top_negative_airlines)

Companhias aéreas mais frequentes:
Index(['Interjet', 'Frontier Airlines', 'Spirit Airlines'], dtype='object', name='airline_name')


In [19]:
print("Avaliações negativas: ", len(df[(df['airline_name'].isin(top_negative_airlines)) & (df['rating_category_numeric'] == -1)]))
print("Avaliações positivas: ", len(df[(df['airline_name'].isin(top_negative_airlines)) & (df['rating_category_numeric'] == 1)]))
print("Avaliações neutras: ", len(df[(df['airline_name'].isin(top_negative_airlines)) & (df['rating_category_numeric'] == 0)]))

Avaliações negativas:  293
Avaliações positivas:  3
Avaliações neutras:  4


In [20]:
nps_result(top_negative_airlines, df)

           Companhia      Atraso     NPS
0           Interjet  Sem Atraso  -93.33
1           Interjet  Com Atraso -100.00
2  Frontier Airlines  Sem Atraso  -88.00
3  Frontier Airlines  Com Atraso -100.00
4    Spirit Airlines  Sem Atraso  -88.68
5    Spirit Airlines  Com Atraso -100.00


### Seleção de 3 companhias aéreas - avaliação +positivas

In [21]:
top_positive_airlines = df[df.rating_category=='Positivo']['airline_name'].value_counts().head(3).index

print("Companhias aéreas mais frequentes:")
print(top_positive_airlines)

Companhias aéreas mais frequentes:
Index(['China Southern Airlines', 'QantasLink', 'Bulgaria Air'], dtype='object', name='airline_name')


In [22]:
nps_result(top_positive_airlines, df)

                 Companhia      Atraso    NPS
0  China Southern Airlines  Sem Atraso  61.04
1  China Southern Airlines  Com Atraso  66.67
2               QantasLink  Sem Atraso  55.10
3               QantasLink  Com Atraso  30.43
4             Bulgaria Air  Sem Atraso  34.69
5             Bulgaria Air  Com Atraso -15.79


In [23]:
print("Avaliações negativas: ", len(df[(df['airline_name'].isin(top_positive_airlines)) & (df['rating_category_numeric'] == -1)]))
print("Avaliações positivas: ", len(df[(df['airline_name'].isin(top_positive_airlines)) & (df['rating_category_numeric'] == 1)]))
print("Avaliações neutras: ", len(df[(df['airline_name'].isin(top_positive_airlines)) & (df['rating_category_numeric'] == 0)]))

Avaliações negativas:  101
Avaliações positivas:  158
Avaliações neutras:  35
